In [1]:
import random

In [2]:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Version/14.0.3 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"
]
def get_random_headers():
    user_agent = random.choice(USER_AGENTS)
    headers = {
        "User-Agent": user_agent,
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Referer": "https://www.ssrn.com/index.cfm/en/",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    }
    return headers

In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import re
import random

# 初始化 WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-infobars")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

# 打开目标网站
url = "https://papers.ssrn.com/sol3/DisplayAbstractSearch.cfm"
driver.get(url)

try:
    # 等待“Accept Cookies”按钮加载
    try:
        accept_cookies_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='接受Cookies']"))
        )
        accept_cookies_button.click()
        print("已点击接受 Cookies 按钮")
    except Exception:
        print("未找到 Cookies 按钮，跳过此步骤")

    # 等待页面加载完成
    time.sleep(3)

    # 填写搜索表单
    search_box = driver.find_element(By.NAME, "advanced_search")
    search_box.send_keys("Machine Learning")

    # 选择搜索时间范围
    date_dropdown = Select(driver.find_element(By.ID, "optionDateLimitId"))
    date_dropdown.select_by_value("5")  # 选择过去一年

    # 点击搜索按钮
    search_box.send_keys(Keys.RETURN)

    # 检测是否出现 CAPTCHA
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "captcha-class"))  # 替换为实际的 CAPTCHA 选择器
        )
        input("请手动完成 CAPTCHA 验证，然后按 Enter 继续...")
    except Exception:
        print("未检测到 CAPTCHA，继续爬取...")

    # 等待搜索结果加载
    n = random.randint(10, 15)
    WebDriverWait(driver, n).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.title.optClickTitle"))
    )

    # 提取搜索结果
    results = driver.find_elements(By.CSS_SELECTOR, "a.title.optClickTitle")
    print(f"找到 {len(results)} 篇论文")

    # 逐一提取论文的标题、链接、作者和 ID
    for result in results:
        title = result.text
        link = result.get_attribute("href")

        # 从链接中提取论文 ID
        paper_id = re.search(r"abstract_id=(\d+)", link)
        paper_id = paper_id.group(1) if paper_id else "未知"
        
        # 提取 `<i>` 标签中的详细描述信息
        details_element = result.find_element(By.CSS_SELECTOR, "div > i")
        details_text = details_element.text.strip()

        # # 打开论文链接，提取作者信息
        # driver.execute_script("window.open(arguments[0]);", link)
        # n = random.randint(10, 15)
        # WebDriverWait(driver, n).until(
        #     lambda d: len(d.window_handles) > 1
        # )  # 等待新标签页打开
        # driver.switch_to.window(driver.window_handles[-1])  # 切换到新标签页
        # 
        # # try:
        # #     # 等待并提取作者信息
        # #     meta_tags = driver.find_elements(By.CSS_SELECTOR, "meta[name='citation_author']")
        # #     authors = [meta.get_attribute("content") for meta in meta_tags]
        # # except Exception:
        # #     authors = ["未知作者"]

        # 打印论文信息
        print(f"标题: {title}")
        print(f"链接: {link}")
        print(f"ID: {paper_id}")
        print(f"描述: {details_text}")

finally:
    # 关闭浏览器
    driver.quit()

已点击接受 Cookies 按钮
未检测到 CAPTCHA，继续爬取...
找到 56 篇论文


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div > i"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010f16eab8 chromedriver + 7092920
1   chromedriver                        0x000000010f16639a chromedriver + 7058330
2   chromedriver                        0x000000010eb0cd70 chromedriver + 400752
3   chromedriver                        0x000000010eb5bbc3 chromedriver + 723907
4   chromedriver                        0x000000010eb5bec1 chromedriver + 724673
5   chromedriver                        0x000000010eb503b6 chromedriver + 676790
6   chromedriver                        0x000000010eb8024d chromedriver + 873037
7   chromedriver                        0x000000010eb502b8 chromedriver + 676536
8   chromedriver             

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import re

# 初始化 WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-infobars")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

# 打开目标网站
url = "https://papers.ssrn.com/sol3/DisplayAbstractSearch.cfm"
driver.get(url)

try:
    # 等待“Accept Cookies”按钮加载
    try:
        accept_cookies_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='接受Cookies']"))
        )
        accept_cookies_button.click()
        print("已点击接受 Cookies 按钮")
    except Exception:
        print("未找到 Cookies 按钮，跳过此步骤")

    # 等待页面加载完成
    time.sleep(3)
    
    # 填写搜索表单
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "advanced_search"))
    )
    search_box.send_keys("Machine Learning")

    # 选择时间范围
    date_dropdown = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "optionDateLimitId"))
    ))
    date_dropdown.select_by_value("5")  # 过去一年

    # 点击搜索按钮
    search_box.send_keys(Keys.RETURN)

    # 检测是否出现 CAPTCHA
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "captcha-class"))  # 替换为实际的 CAPTCHA 选择器
        )
        input("请手动完成 CAPTCHA 验证，然后按 Enter 继续...")
    except Exception:
        print("未检测到 CAPTCHA，继续爬取...")

    # 初始化 CSV 文件
    with open("ssrn_results.csv", "w", encoding="GBK") as file:
        file.write("标题,链接,ID,作者\n")  # 写入表头

    # 设置页数计数器
    page_count = 0
    max_pages = 3  # 最大页数限制

    while page_count < max_pages:
        # 等待搜索结果加载
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.trow.abs"))
        )

        # 提取搜索结果
        results = driver.find_elements(By.CSS_SELECTOR, "div.trow.abs")
        print(f"第 {page_count + 1} 页找到 {len(results)} 条结果")

        for result in results:
            try:
                # 提取标题、链接和 ID
                title_element = result.find_element(By.CSS_SELECTOR, "a.title.optClickTitle")
                title = title_element.text.strip().replace(",", "，")
                link = title_element.get_attribute("href")
                ID = re.search(r"abstract_id=(\d+)", link).group(1)
                
                # 提取作者名字（不含链接）
                authors_div = result.find_element(By.CSS_SELECTOR, "div.authors-list")
                authors = authors_div.find_elements(By.TAG_NAME, "a")
                author_names = [author.text.strip() for author in authors]  # 只提取作者名字
                authors_str = "; ".join(author_names)  # 用分号分隔多个作者

                # 保存结果到 CSV 文件
                with open("ssrn_results.csv", "a", encoding="GBK") as file:
                    file.write(f"{title},{link},{ID},{authors_str}\n")
                
            except Exception as e:
                print(f"解析结果时出错: {e}")

        # 增加页数计数器
        page_count += 1

        # 检查是否有下一页按钮
        if page_count < max_pages:
            try:
                next_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "Next"))  # 替换为实际的“下一页”按钮文字或选择器
                )
                next_button.click()
                print(f"已点击下一页，第 {page_count + 1} 页")
                time.sleep(3)  # 等待页面加载
            except Exception:
                print("未找到下一页按钮，爬取结束")
                break

finally:
    # 关闭浏览器
    driver.quit()

已点击接受 Cookies 按钮
未检测到 CAPTCHA，继续爬取...
第 1 页找到 50 条结果
已点击下一页，第 2 页
第 2 页找到 50 条结果
已点击下一页，第 3 页
第 3 页找到 50 条结果
解析结果时出错: 'gbk' codec can't encode character '\xf6' in position 125: illegal multibyte sequence
解析结果时出错: 'gbk' codec can't encode character '\u015a' in position 205: illegal multibyte sequence
解析结果时出错: 'gbk' codec can't encode character '\u015a' in position 187: illegal multibyte sequence
